# Лицензия

MIT License

Copyright (c) 2023 Victoria Firsanova

Permission is hereby granted, free of charge, to any person obtaining a copy
of this software and associated documentation files (the "Software"), to deal
in the Software without restriction, including without limitation the rights
to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
copies of the Software, and to permit persons to whom the Software is
furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all
copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE
SOFTWARE.

# Импорт библиотек

In [1]:
from datetime import date, timedelta

import requests
from bs4 import BeautifulSoup

import json

# Генерация ссылок

1. Генерация списка дат

https://lenta.ru/2023/09/15/

делим ссылку на условно две части: хттп лента ру и дата

In [2]:
def generate_dates(start_date, end_date):
  """
  Функция принимает на вход даты для генерации списка дат, возвращает список дат.
  start_date и end_date являются классами типа date из модуля datetime
  start_date: date()
  end_date: date()
  """

  # список дат

  dates = []

  # с помощью timedelta мы задаем какую разницу во времени мы отсчитываем
  # поскольку нас интересуют статьи за каждый день, мы отсчитываем 1 день от каждой заданной даты
  # например, мы берем дату 24.07.2021 в качестве начальной точки отсчета
  # затем мы используем дельту равную одному дню и получаем дату 25.07.2021

  delta = timedelta(days = 1) # задаем частоту

  # цикл: до тех пор, пока первоначальная дата не совпадет с (меньше или равна) финальной
  while start_date <= end_date:
    # добавляем текущую (стартовую) дату в список дат
    dates.append(start_date)
    # используем timedelta: прибавляем 1 день к текущей (стартовой) дате
    start_date += delta

  # функция возвращает список дат

  return dates

In [4]:
# объекты типа date оперирует датами в формате (год, месяц, день)
# задаем даты

start_date = date(2023, 9, 22)
end_date = date(2023, 9, 24)

# применяем нашу функцию и смотрим результат

dates = generate_dates(start_date, end_date)
dates

[datetime.date(2023, 9, 22),
 datetime.date(2023, 9, 23),
 datetime.date(2023, 9, 24)]

2. Генерация ссылок для запроса по архивам

In [5]:
def generate_links(website, dates):
  """
  Функция для генерации ссылок. Принимает на вход названия новостных сайтов и
  список ссылок.

  website: str
  dates: list
  """
  # создаем список для хранения ссылок
  links = []
  # создаем временное хранилище для ссылок, которые мы будем записывать в список
  temp = ''

  # начинаем итерацию по всем элементам списка dates
  for i in range(len(dates)):
    # поспользуемся форматированием строки для компляции ссылок
    # используем нашу структуру https://***.ru/news/yyyy/mm/dd
    # форматированию подвергаются элементы ***, yyyy/mm/dd
    # на место *** помещаем название сайта, записанное в переменную website
    # на место yyyy/mm/dd помещаем значения из списка dates текущей итерации
    # вспоминаем, как делать запрос к формату dates (см. ячейку выше)
    # так, чтобы получить значение года, вводим dates[i].year, где
    # > dates - обращение к списку дат
    # > [i] - текущая итерация
    # > year - извлечение года из формата date модуля datetime
    year = dates[i].year
    # к значению месяца нужно приписать 0 слева в том случае,
    # если это значение меньше 10
    # иначе мы не сможем сделать запросы к сайтам,
    # потому что на сайтах используется формат вида 24-_0_7-2023
    month = dates[i].month if dates[i].month > 9 else f'0{dates[i].month}'
    # то же самое нужно сделать для дней
    day = dates[i].day if dates[i].day > 9 else f'0{dates[i].day}'
    temp = f'https://{website}.ru/news/{year}/{month}/{day}'
    links.append(temp)
  return links

In [7]:
# генерация списка ссылок
# добавим название сайта и дату

links = generate_links("lenta", dates)
links

['https://lenta.ru/news/2023/09/22',
 'https://lenta.ru/news/2023/09/23',
 'https://lenta.ru/news/2023/09/24']

# Выгрузка архивов

In [14]:
sample = "https://lenta.ru/articles/2023/09/15/nyfashionweek/"
r = requests.get(sample)
r

<Response [200]>

In [15]:
soup = BeautifulSoup(r.content, 'html.parser')

выдает исходный код страницы

In [ ]:
soup

выдает текст страницы

In [ ]:
soup.text

In [18]:
json.loads(soup.find('script', type='application/ld+json').text)['description']

'Завершилась первая из четырех Недель мод — дизайнеры со всего мира представили публике свои коллекции сезона весна-лето 2024. Несмотря на то что превалирующей тематикой шоу остается нагота, модельеры нашли новые способы удивить своих зрителей. Чем запомнилась Неделя моды в Нью-Йорке — в галерее «Ленты.ру». '

In [19]:
def scraping(links):
  """
  Функция для скрейпинга веб-страниц.
  Используется как для извлечения информации из архивов,
  так и для извлечения информации с новостных страниц сайтов.

  На вход принимается:
  links: список ссылок (list)

  Функция возвращает список собранных данных:
  data: list
  """
  # создаем хранилище для собранной информации
  data = []
  # извлекаем ссылку из нашего списка с помощью итерирования
  for link in links:
    try: #исключаем все ошибки
      # делаем запрос к странице с помощью requests.get
      r = requests.get(link)
      # проверяем успешность запроса
      status = r.status_code
      # если запрос успешный, то есть возвращается значение 200
      if status == 200:
        # производим парсинг страницы с помощью HTML-парсера BeautifulSoup
        soup = BeautifulSoup(r.content, 'html.parser')
        # сохраняем в хранилище
        data.append(soup)
        # для дебагинга кода будем возвращать статус загрузки
        print(f'Успешно выгружены данные со страницы {link}')
      # если значение запроса иное, например, 404
      else:
        # возвращаем сообщение об ошибке
        print(f'При попытке запроса произошла ошибка. Код {r.status_code}')
    # прописываем действия на тот случай, если сделать запрос не удалось
    except requests.exceptions.RequestException as e:
      # возвращаем сообщение об ошибке
      print(f'При попытке запроса произошла ошибка {e}')
  # при успешном прохождении всех ссылок, получаем наши данные
  return data

In [20]:
# последовательно применяем нашу функцию к ссылкам

archive = scraping(links)

Успешно выгружены данные со страницы https://lenta.ru/news/2023/09/22
Успешно выгружены данные со страницы https://lenta.ru/news/2023/09/23
Успешно выгружены данные со страницы https://lenta.ru/news/2023/09/24


чтобы посмотреть весь архив

In [ ]:
archive [0]

# Извлечение новостных страниц из архивов

In [27]:
# применим ту же функцию для пары тестовых страниц

text_url = ["https://lenta.ru/articles/2023/09/15/nyfashionweek/", "https://lenta.ru/articles/2023/09/15/soundoffreedom/"]
sample = scraping(text_url)

Успешно выгружены данные со страницы https://lenta.ru/articles/2023/09/15/nyfashionweek/
Успешно выгружены данные со страницы https://lenta.ru/articles/2023/09/15/soundoffreedom/


In [28]:
# предварительно зададим функцию для формирования ссылок

def form_href(element):
  """
  Функция принимает на вход элемент списка
  из временного хранилища информации с архива сайта lenta (BeautifulSoup).

  Функция возвращает сформированную ссылку для скрейпинга (str).
  """
  # извлекаем из элемента ссылку
  # более подробно работа с парсингом описана выше
  href = element['href'] # пример извлечения: /news/2021/07/24/***/
  # собираем ссылку
  # префикс 'https://lenta.ru' соединяем с извлеменной ссылкой
  return 'https://lenta.ru' + href

json = dictionary

мы храним его в фигурных скобках; имя словаря состоит из двух элементов: ключ и значение

In [45]:
def get_content(contents):
  temp = json.loads(contents.find('script', type='application/ld+json').text)
  content = {'headline': temp['headline'], 'description': temp['description'], 'body': temp['articleBody']}
  return content

In [46]:
# создаем хранилище для новостного контента

content = []

In [47]:
# проходим по каждому элементу архива,
# то есть по данным за каждый день
for page in archive:
  # находим все элементы класса "card-full-news _archive"
  # методом find_all из BeautifulSoup
  # ранее мы выяснили, что именно в этом классе хранятся искомые ссылки
  # в переменной temp хранится список (list) всех элементов класса "card-full-news _archive"

  temp = page.find_all("a", class_="card-full-news _archive") #класс хранит ссылки на страницы в архиве

  # проходим по всем элементам этого списка
  # здесь используем созданную ранее функцию lenta_href для создания ссылок
  # с помощью генератора списков создаем набор ссылок для каждого дня в архиве

  temp_links = [form_href(element) for element in temp] #склеивает страницу и ссылку на нее

  # используем созданную ранее функцию scraping для скрейпинга данных
  # по этим ссылкам

  contents = scraping(temp_links) #достает исходный код страницы

  # добавляем извлеченную информацию в хранилище content
  # contents хранит список данных за по каждой ссылке
  # эти данные представляют собой объект BeautifulSoup
  # с помощью get_content мы извлекаем текстовые данные из каждого элемента
  # и с помощью генерации списков мы все это сохраняем в хранилище content

  content.append([get_content(c) for c in contents]) #извлекает из зашифрованного скрипта информацию

Успешно выгружены данные со страницы https://lenta.ru/news/2023/09/21/vzryv/
Успешно выгружены данные со страницы https://lenta.ru/news/2023/09/22/way/
Успешно выгружены данные со страницы https://lenta.ru/news/2023/09/22/358/
Успешно выгружены данные со страницы https://lenta.ru/news/2023/09/22/meet/
Успешно выгружены данные со страницы https://lenta.ru/news/2023/09/21/abrams/
Успешно выгружены данные со страницы https://lenta.ru/news/2023/09/22/kolomoisky_aktivi/
Успешно выгружены данные со страницы https://lenta.ru/news/2023/09/22/toplesgrey/
Успешно выгружены данные со страницы https://lenta.ru/news/2023/09/22/paket/
Успешно выгружены данные со страницы https://lenta.ru/news/2023/09/22/mirzoyan/
Успешно выгружены данные со страницы https://lenta.ru/news/2023/09/22/serb/
Успешно выгружены данные со страницы https://lenta.ru/news/2023/09/22/moldova/
Успешно выгружены данные со страницы https://lenta.ru/news/2023/09/22/help/
Успешно выгружены данные со страницы https://lenta.ru/news/2

In [49]:
len(content)

3

In [50]:
len(content [0])

30

In [51]:
content [0][2]

{'headline': 'В США рассказали о показанной Шойгу в Иране «странной» ракете',
 'description': 'Министру обороны России Сергей Шойгу в ходе визита в Тегеран показали «странную» барражирующую ракету класса «земля-воздух» иранской разработки, известную в США как 358. Об этом рассказывает The Drive. Издание пишет, что информации о 358 и ее возможностях в открытых источниках мало.',
 'body': 'Министру обороны России Сергей Шойгу в ходе визита в Тегеран показали «странную» барражирующую ракету класса «земля-воздух» иранской разработки, известную в США как 358. Об этом рассказывает американское издание The Drive. Отмечается, что образцы этого оружия, о котором в США впервые узнали в 2019 году, Иран поставляет своим союзникам в Йемене и Ираке. Издание пишет, что информации о 358 и ее возможностях в открытых источниках мало. В частности, отмечается, что 2,4-метровая ракета цилиндрической формы имеет три группы крыльев, предназначенных для маневрирования и планирования в полете, и оснащена инерц

сохраняем дата сеты:

1 чтобы они хорошо читались человеком
2 хорошо читались машиной

для этого хорошо подходят табличные форматы csv (данные через запятую) или tsv (данные через табуляцию)



In [52]:
# сохраним наш корпус в отдельный файл
with open("corpus.json", "w", encoding="utf8") as f:
   json.dump(content, f, ensure_ascii=False)

Задание:

- создать функцию, которая принимает на вход даты и возвращает корпус
- функция должна line-by-line сохранять все в файл
- "поиграть" с разметкой корпуса
- сохранить в другом формате (csv, txt)
- собрать корпус для другого издания
- собрать более крупный корпус